<table><tbody><tr><th><p><img alt="Emblema" src="https://cdn6.aptoide.com/imgs/6/f/4/6f4821daa840da8fe971445350759fe5_icon.png" style="width:150px;"></p></th><th><p><strong>Inteligencia Artificial</strong></p><p><strong>Grado en Ingeniería Informática en Sistemas de Información – Curso 2024/2025</strong></p><p><strong>ENSEÑANZAS PRÁCTICAS Y DE DESARROLLO</strong></p><h1>EPD 6: Machine Learning – Sistemas de recomendación</h1></th></tr></tbody></table>

____

## Objetivos
- Implementación en Python de un algoritmo de sistemas de recomendación.

___

## Bibliografía Básica
- Recommender systems. Charu C. Aggarwal. Springer, 2016. Disponible online: http://pzs.dstu.dp.ua/DataMining/recom/bibl/1aggarwal_c_c_recommender_systems_the_textbook.pdf

- Recommender systems handbook. Francesco Ricci, Lior Rokach, Bracha Shapira, Paul B. Kantor. Springer, 2011. Disponible online: https://www.cse.iitk.ac.in/users/nsrivast/HCC/Recommender_systems_handbook.pdf

___

In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.optimize as opt

## Ejercicios
Implementar un algoritmo que recomiende películas a los usuarios. Para ello, usar el fichero “ex8_movies.mat” que contiene datos de películas clasificadas por los usuarios en una escala del 1 al 5. En concreto, 943 usuarios han clasificado 1682 películas. Las películas se identifican con 10 características relativas a su contenido. El objetivo del algoritmo es predecir la puntuación que le daría un usuario a una película que no ha visto aún y recomendar a ese usuario las películas con las puntuaciones más altas.

#### EJ01. 

Cargar el dataset y prepararlo para el algoritmo usando 2 matrices. La matriz Y almacenará las clasificaciones de las películas y la matriz R contendrá solamente valores binarios donde R(i,j) = 1 significará que el usuario j clasificó la película i y R(i,j) = 0 indicará que no la clasificó. Ambas matrices tendrán como dimensión: número de películas x número de usuarios. La media de las puntuaciones que recibe la primera película (Toy Story) debe ser aproximadamente 3.878319. Almacenar en las matrices de parámetros X y Theta los valores pre-entrenados disponibles en el fichero “ex8_movieParams.mat”. Las dimensiones de X deben ser número de películas x número de características y las de Theta número de características x número de usuarios. Compruebe las dimensiones y actúe en caso de que no coincidan.

##### Solución:

In [2]:
# =============== EJ1: Cargar datos ================
print('Loading movie ratings dataset.')
movies = sio.loadmat("ex8_movies.mat")
Y = movies['Y'] # [n_items, n_users] puntuaciones de 1-5
R = movies['R'] # [n_items, n_users] R(i,j)=1 si usuario j puntuó pelicula i
print("Shape de Y: ", Y.shape)  # [n_items, features]
print("Shape de R: ", R.shape)  # [n_items, features]

print('\tAverage rating for the first movie (Toy Story): ', Y[0, np.where(R[0, :] == 1)[0]].mean(), "/5\n")

#  Cargar parámetros preentrenados (X, Theta, num_users, num_movies, num_features)
    
params_data = sio.loadmat('ex8_movieParams.mat')
X = params_data['X']
Theta = params_data['Theta'] 
Theta = Theta.T
print("Shape de X: ", X.shape)  # [n_items, features]
print("Shape de Theta: ", Theta.shape)  # [features, n_users]


Loading movie ratings dataset.
Shape de Y:  (1682, 943)
Shape de R:  (1682, 943)
	Average rating for the first movie (Toy Story):  3.8783185840707963 /5

Shape de X:  (1682, 10)
Shape de Theta:  (10, 943)


#### EJ02.
Implementar la función coste sin regularización para un sistema de recomendación de filtrado colaborativo en cofiCostFuncSinReg siguiendo la fórmula indicada en EB. El coste se acumula para el usuario j y la película i sólo si R(i,j)= 1. Si usa las matrices de parámetros X y Theta almacenadas en el fichero para los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos/características, el coste debe ser 22.22 aproximadamente.

##### Solución:

In [3]:
def cofiCostFuncSinReg(params, Y, R, num_features):
    nPeliculas = Y.shape[0]
    nUsuarios = Y.shape[1]
    X = np.reshape(params[:nPeliculas * num_features], (nPeliculas, num_features), 'F')
    Theta = np.reshape(params[nPeliculas * num_features:], (num_features, nUsuarios), 'F')
    J = (1/2)*np.sum(np.power(np.multiply(np.dot(X, Theta) - Y, R),2))
    return J

def cofiCostFuncReg(params, Y, R, num_features, lambda_param):
    nPeliculas = Y.shape[0]
    nUsuarios = Y.shape[1]
    X = np.reshape(params[:nPeliculas * num_features], (nPeliculas, num_features), 'F')
    Theta = np.reshape(params[nPeliculas * num_features:], (num_features, nUsuarios), 'F')
    J = (1/2)*np.sum(np.power(np.multiply(np.dot(X, Theta) - Y, R), 2))
    J += (lambda_param/2) * (np.sum(np.square(Theta)) + np.sum(np.square(X)))
    return J

In [4]:
### Subconjunto de datos para que ejecute más rápidamente
users = 4
movies = 5
features = 3

X_sub = X[:movies, :features]
Theta_sub = Theta[:features, :users]
Y_sub = Y[:movies, :users]
R_sub = R[:movies, :users]

params = np.hstack((np.ravel(X_sub, order='F'), np.ravel(Theta_sub, order='F'))) # Desenrollar: primero X_sub luego Theta_sub
J = cofiCostFuncSinReg(params, Y_sub, R_sub, features)
print("Cost without regularization at loaded parameters: ", J, "(this value should be about 22.22)")


Cost without regularization at loaded parameters:  22.224603725685675 (this value should be about 22.22)


#### EJ03.
Implementar la función gradiente sin regularización en cofiGradientFuncSinReg. Usar la función auxiliar checkNNGradientsSinReg.py para verificar que los gradientes están bien calculados.

##### Solución:

In [5]:
def cofiGradientFuncSinReg(params, Y, R, num_features):
    nPeliculas = Y.shape[0]
    nUsuarios = Y.shape[1]
    X = np.reshape(params[:nPeliculas * num_features], (nPeliculas, num_features), 'F')
    Theta = np.reshape(params[nPeliculas * num_features:], (num_features, nUsuarios), 'F')
        
    error = np.multiply(np.dot(X, Theta) - Y, R)
    
    ThetaGrad = np.dot(X.T, error)
    XGrad = np.dot(error, Theta.T)
    
    grad = np.hstack((np.ravel(XGrad, order='F'), np.ravel(ThetaGrad, order='F')))
    
    return grad

In [6]:
def computeNumericalGradientSinReg(X,Theta, Y, R, num_features):
    mygrad = np.zeros(Theta.size + X.size)
    perturb = np.zeros(Theta.size + X.size)
    myeps = 0.0001
    params = np.concatenate((np.ravel(X, order='F'), np.ravel(Theta, order='F')))

    for i in range(np.size(Theta)+np.size(X)):
        # Set perturbation vector
        perturb[i] = myeps
        params_plus = params + perturb
        params_minus = params - perturb
        cost_high = cofiCostFuncSinReg(params_plus, Y, R, num_features)
        cost_low = cofiCostFuncSinReg(params_minus, Y, R, num_features)

        # Compute Numerical Gradient
        mygrad[i] = (cost_high - cost_low) / float(2 * myeps)
        perturb[i] = 0

    return mygrad
    
def checkNNGradientsSinReg():
    #Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    #Zap out most entries
    Y = X_t @ Theta_t.T
    dim = Y.shape
    aux = np.random.rand(*dim)
    Y[aux > 0.5] = 0
    R = np.zeros((Y.shape))
    R[Y != 0] = 1

    #Run Gradient Checking
    dim_X_t = X_t.shape
    dim_Theta_t = Theta_t.shape
    X = np.random.randn(*dim_X_t)
    Theta = np.random.randn(*dim_Theta_t)
    num_users = Y.shape[1]
    num_movies = Y.shape[0]
    num_features = Theta_t.shape[1]

    params = np.concatenate((np.ravel(X,order='F'), np.ravel(Theta,order='F')))

    # Calculo gradiente mediante aproximación numérica
    mygrad = computeNumericalGradientSinReg(X, Theta, Y, R, num_features)

    #Calculo gradiente
    grad = cofiGradientFuncSinReg(params, Y, R, num_features)

    # Visually examine the two gradient computations.  The two columns
    # you get should be very similar.
    df = pd.DataFrame(mygrad,grad)
    print(df)

    # Evaluate the norm of the difference between two solutions.
    # If you have a correct implementation, and assuming you used EPSILON = 0.0001
    # in computeNumericalGradient.m, then diff below should be less than 1e-9
    diff = np.linalg.norm((mygrad-grad))/np.linalg.norm((mygrad+grad))

    print('If your gradient implementation is correct, then the differences will be small (less than 1e-9):' , diff)
    

In [7]:
grad = cofiGradientFuncSinReg(params, Y_sub, R_sub, features)
print("Gradient without regularization at loaded parameters: \n", grad)

checkNNGradientsSinReg()


Gradient without regularization at loaded parameters: 
 [ -2.52899165  -0.56819597  -0.83240713  -0.38358278  -0.80378006
   7.57570308   3.35265031   4.91163297   2.26333698   4.74271842
  -1.89979026  -0.52339845  -0.76677878  -0.35334048  -0.74040871
 -10.5680202    4.62776019  -7.16004443  -3.05099006   1.16441367
  -3.47410789   0.           0.           0.           0.
   0.           0.        ]
                  0
-1.559586 -1.559586
 0.919743  0.919743
-2.705700 -2.705700
-4.806545 -4.806545
 2.705937  2.705937
-4.532276 -4.532276
 4.445120  4.445120
 8.748818  8.748818
 1.191697  1.191697
 0.234830  0.234830
 2.142682  2.142682
 2.427612  2.427612
 4.822360  4.822360
-2.755603 -2.755603
-5.042536 -5.042536
-2.012012 -2.012012
 1.244494  1.244494
-0.313439 -0.313439
 0.064707  0.064707
-0.050027 -0.050027
-0.092127 -0.092127
 0.040891  0.040891
-0.022455 -0.022455
 0.040383  0.040383
-0.532557 -0.532557
 0.411738  0.411738
 0.758229  0.758229
If your gradient implementation is

#### EJ04.
Implementar la función coste y la función gradiente con regularización en cofiCostFuncReg y cofiGradientFuncReg respectivamente. Se debe incluir el parámetro lambda inicializado a 1.5. La función coste debe devolver un coste de 31.34 aproximadamente si usa las matrices de parámetros X y Theta almacenadas en el fichero “ex8_movieParams.mat” para los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos. Usar la función auxiliar checkNNGradientsReg con el parámetro lambda inicializado a 1.5 para verificar que los gradientes están bien calculados.

##### Solución:

In [8]:
def cofiGradientFuncReg(params, Y, R, num_features, lambda_param):
    nPeliculas = Y.shape[0]
    nUsuarios = Y.shape[1]
    X = np.reshape(params[:nPeliculas * num_features], (nPeliculas, num_features), 'F')
    Theta = np.reshape(params[nPeliculas * num_features:], (num_features, nUsuarios), 'F')
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    ThetaGrad = np.dot(X.T, error) + (lambda_param*Theta)
    XGrad = np.dot(error, Theta.T) + (lambda_param*X)
    
    grad = np.hstack((np.ravel(XGrad, order='F'), np.ravel(ThetaGrad, order='F')))
    return grad


In [9]:
def computeNumericalGradientReg(X,Theta, Y, R, num_features, lambda_param):
    mygrad = np.zeros(Theta.size + X.size)
    perturb = np.zeros(Theta.size + X.size)
    myeps = 0.0001
    params = np.concatenate((np.ravel(X, order='F'), np.ravel(Theta, order='F')))

    for i in range(np.size(Theta)+np.size(X)):
        # Set perturbation vector
        perturb[i] = myeps
        params_plus = params + perturb
        params_minus = params - perturb
        cost_high = cofiCostFuncReg(params_plus, Y, R, num_features, lambda_param)
        cost_low = cofiCostFuncReg(params_minus, Y, R, num_features, lambda_param)

        # Compute Numerical Gradient
        mygrad[i] = (cost_high - cost_low) / float(2 * myeps)
        perturb[i] = 0

    return mygrad
    
def checkNNGradientsReg(lambda_param):
    #Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    #Zap out most entries
    Y = X_t @ Theta_t.T
    dim = Y.shape
    aux = np.random.rand(*dim)
    Y[aux > 0.5] = 0
    R = np.zeros((Y.shape))
    R[Y != 0] = 1

    #Run Gradient Checking
    dim_X_t = X_t.shape
    dim_Theta_t = Theta_t.shape
    X = np.random.randn(*dim_X_t)
    Theta = np.random.randn(*dim_Theta_t)
    num_users = Y.shape[1]
    num_movies = Y.shape[0]
    num_features = Theta_t.shape[1]

    params = np.concatenate((np.ravel(X,order='F'), np.ravel(Theta,order='F')))

    # Calculo gradiente mediante aproximación numérica
    mygrad = computeNumericalGradientReg(X, Theta, Y, R, num_features, lambda_param)

    #Calculo gradiente
    grad = cofiGradientFuncReg(params, Y, R, num_features, lambda_param)

    # Visually examine the two gradient computations.  The two columns
    # you get should be very similar.
    df = pd.DataFrame(mygrad,grad)
    print(df)

    # Evaluate the norm of the difference between two solutions.
    # If you have a correct implementation, and assuming you used EPSILON = 0.0001
    # in computeNumericalGradient.m, then diff below should be less than 1e-9
    diff = np.linalg.norm((mygrad-grad))/np.linalg.norm((mygrad+grad))

    print('If your gradient implementation is correct, then the differences will be small (less than 1e-9):' , diff)
    

In [10]:
# Evaluate cost function and gradient function, both with regularization
lambda_param = 1.5
J = cofiCostFuncReg(params, Y_sub, R_sub, features, lambda_param)
print("\n\nCost with regularization at loaded parameters: ", J, "(this value should be about 31.34)")

grad = cofiGradientFuncReg(params, Y_sub, R_sub, features, lambda_param)
print("Gradient with regularization at loaded parameters: \n", grad)
checkNNGradientsReg(lambda_param)




Cost with regularization at loaded parameters:  31.34405624427422 (this value should be about 31.34)
Gradient with regularization at loaded parameters: 
 [ -0.95596339   0.60308088   0.12985616   0.29684395   0.60252677
   6.97535514   2.77421145   4.0898522    1.06300933   4.90185327
  -0.10861109   0.25839822  -0.89247334   0.66738144  -0.19747928
 -10.13985478   2.10136256  -6.76563628  -2.29347024   0.48244098
  -2.99791422  -0.64787484  -0.71820673   1.27006666   1.09289758
  -0.40784086   0.49026541]
                    0
 0.912839    0.912839
-3.232312   -3.232312
-0.484110   -0.484110
-4.566465   -4.566465
 6.097694    6.097694
 2.679610    2.679610
-4.385898   -4.385898
-4.551102   -4.551102
-1.507087   -1.507087
-3.836308   -3.836308
 1.403378    1.403378
 3.445399    3.445399
 0.415497    0.415497
-1.799379   -1.799379
 2.563387    2.563387
 1.260677    1.260677
 1.234499    1.234499
 1.628735    1.628735
-0.092972   -0.092972
-0.178646   -0.178646
 0.415238    0.415238
 5

#### EJ05.
Inicializar de forma random con valores pequeños tanto la matriz X como la matriz Theta para todo el conjunto de datos, utilize la función np.random.rand() indicando las dimensiones en los parámetros de entrada. A continuación, entrenar con regularización para obtener los parámetros óptimos X y Theta usando la función fmin_cg de la librería scipy.optimize con 200 iteraciones y lambda con valor 1.5.

##### Solución:

In [11]:
# Useful Values
movies = Y.shape[0]  # 1682
users = Y.shape[1]  # 943
features = 10
lambda_param = 1.5
maxiter = 200

# Inicialización de X y Theta
X = np.random.rand(movies, features) * (2*0.12)
Theta = np.random.rand(features, users) * (2*0.12)
params = np.hstack((np.ravel(X, order='F'), np.ravel(Theta, order='F')))# Desenrollar: primero X luego Theta
# Algoritmo de optimización
fmin_1 = opt.fmin_cg(maxiter=maxiter, f=cofiCostFuncReg, x0=params, fprime=cofiGradientFuncReg,
                  args=(Y, R, features, lambda_param))
# Enrollar el resultado
X_fmin = np.reshape(fmin_1[:movies * features], (movies, features), 'F')
Theta_fmin = np.reshape(fmin_1[movies * features:], (features, users), 'F')


         Current function value: 32978.182602
         Iterations: 200
         Function evaluations: 295
         Gradient evaluations: 295


c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


#### EJ06.
Después del entrenamiento, conseguir la matriz de predicciones. Además, imprimir por pantalla la recomendación de las 10 películas con mejores puntuaciones para el usuario 2. Deben ser películas que no estuviesen previamente puntuadas por dicho usuario, para ello use np.where() con la correspondiente condición.

##### Solución:

In [12]:
predictions = np.dot(X_fmin, Theta_fmin)
# Solo el usuario j
j = 2
res_user = np.zeros((movies, 1))
pred_userj = predictions[:, j] # Seleccionar el usuario j
res_user = np.where(R[:, j] == 0, pred_userj, 0).reshape(-1, 1)
# Para cada película: A las que tenían valor previo le ponemos un 0 y a las que hemos predicho el valor de su predicción
idx = np.argsort(res_user, axis=0)[::-1] # Ordenar por las predicciones de menor a mayor y coger sus índice. [::-1] significa que le damos la vuelta a la salida: es decir lo colocamos de mayor a menor

# Leer el fichero con los nombres de cada película
movie_idx = {}
f = open('movie_ids.txt',encoding = 'ISO-8859-1')
for line in f:
    tokens = line.split(' ')
    tokens[-1] = tokens[-1][:-1]
    movie_idx[int(tokens[0]) - 1] = ' '.join(tokens[1:])
print("Top 10 movie predictions:")
for i in range(10):
    j = int(idx[i])
    print('Predicted rating of {0} for movie {1}.'.format(str(float(res_user[j])), movie_idx[j]))


Top 10 movie predictions:
Predicted rating of 5.35242636292428 for movie Clerks (1994).
Predicted rating of 5.31177711343657 for movie Big Lebowski, The (1998).
Predicted rating of 4.942562290155292 for movie Three Colors: White (1994).
Predicted rating of 4.856147622837372 for movie Flirting With Disaster (1996).
Predicted rating of 4.8403278790650655 for movie Last Supper, The (1995).
Predicted rating of 4.835704659714779 for movie Deceiver (1997).
Predicted rating of 4.659218531210406 for movie Strange Days (1995).
Predicted rating of 4.602072227460943 for movie Hard Eight (1996).
Predicted rating of 4.593308206599591 for movie Trees Lounge (1996).
Predicted rating of 4.5835554313094935 for movie Beavis and Butt-head Do America (1996).


C:\Users\danit\AppData\Local\Temp\ipykernel_18540\1511940430.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  j = int(idx[i])
C:\Users\danit\AppData\Local\Temp\ipykernel_18540\1511940430.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Predicted rating of {0} for movie {1}.'.format(str(float(res_user[j])), movie_idx[j]))
